<a href="https://colab.research.google.com/github/deivismartinez/NER-Medical-Uninorte/blob/main/Inference_mistral_f.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inference Mistral**

In [ ]:
!wget https://zenodo.org/record/4279323/files/meddocan.zip?download=1 -O meddocan.zip

--2025-02-03 22:45:57--  https://zenodo.org/record/4279323/files/meddocan.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.45.92, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/4279323/files/meddocan.zip [following]
--2025-02-03 22:45:58--  https://zenodo.org/records/4279323/files/meddocan.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 11738792 (11M) [application/octet-stream]
Saving to: ‘meddocan.zip’

meddocan.zip        100%[===================>]  11.19M  9.25MB/s    in 1.2s    

2025-02-03 22:45:59 (9.25 MB/s) - ‘meddocan.zip’ saved [11738792/11738792]



In [ ]:
from pathlib import Path
from google.colab import files
from os import listdir
from os.path import isfile, join
import os

In [ ]:
!unzip meddocan.zip

Streaming output truncated to the last 5000 lines.
  inflating: meddocan/test/brat/S0376-78922015000100011-1.ann  
  inflating: meddocan/test/brat/S0376-78922015000100011-1.txt  
  inflating: meddocan/test/brat/S0376-78922016000300005-1.ann  
  inflating: meddocan/test/brat/S0376-78922016000300005-1.txt  
  inflating: meddocan/test/brat/S0376-78922016000300010-1.ann  
  inflating: meddocan/test/brat/S0376-78922016000300010-1.txt  
  inflating: meddocan/test/brat/S0378-48352004000300007-1.ann  
  inflating: meddocan/test/brat/S0378-48352004000300007-1.txt  
  inflating: meddocan/test/brat/S0378-48352005000100005-1.ann  
  inflating: meddocan/test/brat/S0378-48352005000100005-1.txt  
  inflating: meddocan/test/brat/S0378-48352006000300005-1.ann  
  inflating: meddocan/test/brat/S0378-48352006000300005-1.txt  
  inflating: meddocan/test/brat/S0378-48352006000400006-1.ann  
  inflating: meddocan/test/brat/S0378-48352006000400006-1.txt  
  inflating: meddocan/test/brat/S0378-483520060005000

In [ ]:
dataset = {}
for path in (Path('dev'), Path('test'), Path('train')):
    dir_path = Path('meddocan')/path/Path('brat')
    filenames = tuple(f[:-4] for f in listdir(dir_path) if isfile(join(dir_path, f)) if f[-4:] == '.txt')
    dataset[str(path)] = []
    for file_name in filenames:
      d = dict()
      with open(dir_path/Path(file_name+'.txt'), 'r') as f:
        dataset[str(path)].append({"text":f.read(),"file":file_name,"file_name_path":dir_path/Path(file_name+'.txt')})

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
!unzip '/content/mistral_chp80.zip' -d '/content/inference_mistral'

Archive:  /content/mistral_chp80.zip
  inflating: /content/inference_mistral/scheduler.pt  
  inflating: /content/inference_mistral/rng_state.pth  
  inflating: /content/inference_mistral/optimizer.pt  
  inflating: /content/inference_mistral/README.md  
  inflating: /content/inference_mistral/adapter_model.safetensors  
  inflating: /content/inference_mistral/adapter_config.json  
  inflating: /content/inference_mistral/tokenizer.model  
  inflating: /content/inference_mistral/tokenizer.json  
  inflating: /content/inference_mistral/tokenizer_config.json  
  inflating: /content/inference_mistral/special_tokens_map.json  
  inflating: /content/inference_mistral/training_args.bin  
  inflating: /content/inference_mistral/trainer_state.json  


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "inference_mistral", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Mistral patching. Transformers:4.47.0.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

Unsloth 2024.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
def do_inference(text):
  FastLanguageModel.for_inference(model) # Enable native 2x faster inference
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          "You are a Named Entity Recognition in Spanish Language. Do some analysis to extract the Entity from the text for some categories, i.e., Person, Organization, Location, Date/Time, and other as Miscellaneous. Output Person category as PER, Organization category as ORG, Location category as LOC, Date/Time category as DAT, and Miscellaneous category as MISC. Return this result as JSON for each entity with character offset from each result.", # instruction
          text, # input
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 2400, use_cache = True)
  return tokenizer.batch_decode(outputs)

In [ ]:
i=0
for text in dataset['test']:
    i+=1
    print(i)
    value = do_inference(text['text'])
    dir = 'inference_done'
    with open(os.path.join(dir, text['file']+'.txt'), 'w') as file:
      print(value)
      file.writelines(value)

1
['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are a Named Entity Recognition in Spanish Language. Do some analysis to extract the Entity from the text for some categories, i.e., Person, Organization, Location, Date/Time, and other as Miscellaneous. Output Person category as PER, Organization category as ORG, Location category as LOC, Date/Time category as DAT, and Miscellaneous category as MISC. Return this result as JSON for each entity with character offset from each result.\n\n### Input:\nNombre:  Pedro.\nApellidos: De la sierra Rodriguez.\nNHC: 2879056.\nNASS: 43 90871586 95.\nDomicilio: Calle San Juan de Ribera, 2.\nLocalidad/ Provincia: Sevilla.\nCP: 41009.\nDatos asistenciales.\nFecha de nacimiento: 08/08/1993.\nPaís: España.\nEdad: 24 años Sexo: H.\nFecha de Ingreso: 17/05/2018.\nMédico: Daniel Torres Lagares NºCol: 41 41 58796.\nHisto

In [ ]:
import zipfile, os

In [ ]:
def comprimir(folder_name, file_name):
  fantasy_zip = zipfile.ZipFile(file_name, 'w')
  for folder, subfolders, filesFolder in os.walk(folder_name):
      for file in filesFolder:
        fantasy_zip.write(os.path.join(folder, file),
                          os.path.relpath(os.path.join(folder,file), folder_name),
                          compress_type = zipfile.ZIP_DEFLATED)
  fantasy_zip.close()

In [ ]:
comprimir(folder_name='/content/inference_done', file_name='/content/inference_mistral_40.zip')

# **Estructura for metrics - Meddocan Format - Mistral**

In [ ]:
import json

In [ ]:
def do_file_inference(name_file):
  with open(name_file, 'r') as f:
    last_line = f.readlines()[-1]
  result = [last_line[:last_line.index('<|eot_id|>')]]
  labels_list = [json.loads(idx) for idx in result]
  file_out = open('system_llama/'+name_file[name_file.index('inference_llama/')+len('inference_llama/'):], 'w')
  for idx in labels_list[0]:
    #print(idx)
    if 'end' in idx:
      value = idx['entity']+'\t'+str(idx['start'])+'\t'+str(idx['end'])+'\t'+idx['word']+'\n'
    else:
      value = idx['entity']+'\t'+str(idx['start'])+'\t'+str(idx['start']+len(idx['word']))+'\t'+idx['word']+'\n'
    print(value)
    file_out.write(value)
  file_out.close()



In [ ]:
def do_file_meddocan(file):
  file_text = open(file, 'r')
  one_file = file_text.readlines()[-1]
  labels_list = [json.loads(idx) for idx in one_file]
  print(labels_list)
  file_text.close()

In [ ]:
import glob
i = 0
for file in glob.glob("inference_mistral/*.txt"):
  i+=1
  if i==1:
    print(file)
    do_file_meddocan(file)

In [ ]:
import glob
i = 0
for file in glob.glob("inference_llama/*.txt"):

In [ ]:
!unzip '/content/inference_mistral.zip' -d '/content/inference_mistral'

# **Inference Mistral**

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.8: Fast Mistral patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
def do_inference(text):
  FastLanguageModel.for_inference(model) # Enable native 2x faster inference
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          "You are a Named Entity Recognition in Spanish Language. Do some analysis to extract the Entity from the text for some categories, i.e., Person, Organization, Location, Date/Time, and other as Miscellaneous. Output Person category as PER, Organization category as ORG, Location category as LOC, Date/Time category as DAT, and Miscellaneous category as MISC. Return this result as JSON for each entity with character offset from each result.", # instruction
          text, # input
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 5000, use_cache = True)
  return tokenizer.batch_decode(outputs)

In [ ]:
i=0
for text in dataset['test']:
    i+=1
    print(i)
    value = do_inference(text['text'])
    dir = 'inference_done'
    with open(os.path.join(dir, text['file']+'.txt'), 'w') as file:
      print(value)
      file.writelines(value)

# **Formatting **